# <b>Introduction<b>

In this project, I classify Yelp round-10 review datasets. The reviews contain a lot of metadata that can be mined and used to infer meaning, business attributes, and sentiment. For simplicity, I classify the review comments into two class: either as positive or negative. Reviews that have star higher than three are regarded as positive while the reviews with star less than or equal to 3 are negative. Therefore, the problem is a supervised learning. To build and train the model, I first tokenize the text and convert them to sequences. Each review comment is limited to 50 words. As a result, short texts less than 50 words are padded with zeros, and long ones are truncated. After processing the review comments, I trained three model in three different ways: 

<li> Model-1: In this model, a neural network with LSTM and a single embedding layer were used. 
<li> Model-2: In Model-1, an extra 1D convolutional layer has been added on top of LSTM layer to reduce the training time.
<li> Model-3:  In this model, I use the same network architecture as Model-2, but use the pre-trained glove 100 dimension word embeddings as initial input.

Since there are about 1.6 million input comments, it takes a while to train the models. To reduce the training time step, I limit the training epoch to three. After three epochs, it is evident that Model-2 is better regarding both training time and validation accuracy.

## <b>Project Outline <b>

In this project I will cover the follwouings :

<li> Download data from yelp and process them
<li> Build neural network with LSTM
<li> Build neural network with LSTM and CNN
<li> Use pre-trained GloVe word embeddings
<li> Word Embeddings from Word2Vec

## <b>Import libraries<b>

In [48]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

# Other
import pandas as pd
import numpy as np

## <b> Data Processing<b>

In [49]:
df = pd.read_csv('train.csv', sep = '|', names = ['stars', 'text'], error_bad_lines=False)

In [50]:
df= df.dropna()
df = df[df.stars.apply(lambda x: x.isnumeric())]
df = df[df.stars.apply(lambda x: x !="")]
df = df[df.text.apply(lambda x: x !="")]

In [52]:
df.describe()

,stars,text
count,1673870,1673870
unique,5,1673452
top,5,Good stuff
freq,709732,6


In [53]:
df.head()

,stars,text
0,5,The minute I realized that Conflict was a bloc...
2,5,I love Conflict Kitchen. The food is fantasti...
3,4,Holy moly! I'm addicted!\n\nI first heard of C...
4,4,"Had some great Persian food, though it was mor..."
5,4,Yummy food. Good prices. Encourages me to try ...


### Convert five classes into two classes (positive = 1 and negative = 0)

Since the main purpose is to identify positive or negative comments, I convert five class star category into two classes: 

<li> (1) Positive: comments with stars > 3 and 
<li> (2) Negative: comments with stars <= 3

In [54]:
labels = df['stars'].map(lambda x : 1 if int(x) > 3 else 0)

### Tokenize text data

Because of the computational expenses, I use the top 20000 unique words. First, tokenize the comments then convert those into sequences. I keep 50 words to limit the number of words in each comment. 

In [72]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=50)

In [73]:
print(data.shape)

(1673870, 50)


###  <b>Build neural network with LSTM<b>

### Network Architechture

The network starts with an embedding layer. The layer lets the system expand each token to a more massive vector, allowing the network to represent a word in a meaningful way. The layer takes 20000 as the first argument, which is the size of our vocabulary, and 100 as the second input parameter, which is the dimension of the embeddings. The third parameter is the input_length of 50, which is the length of each comment sequence.

In [13]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the network

There are about 1.6 million comments, and it takes a while to train the model in a MacBook Pro. To save time I have used only three epochs. GPU machines can be used to accelerate the training with more time steps. I split the whole datasets as 60% for training and 40% for validation.

In [14]:
model.fit(data, np.array(labels), validation_split=0.4, epochs=3)

Train on 1004322 samples, validate on 669548 samples
Epoch 1/3
1004322/1004322 [==============================] - 7913s - loss: 0.2875 - acc: 0.8776 - val_loss: 0.2553 - val_acc: 0.8934
Epoch 2/3
1004322/1004322 [==============================] - 7931s - loss: 0.2454 - acc: 0.8978 - val_loss: 0.2469 - val_acc: 0.8975
Epoch 3/3
1004322/1004322 [==============================] - 11974s - loss: 0.2291 - acc: 0.9057 - val_loss: 0.2530 - val_acc: 0.8977


NameError: name 'padded_docs' is not defined

##  <b>Build neural network with LSTM and CNN <b>
The LSTM model worked well. However, it takes forever to train three epochs. One way to speed up the training time is to improve the network adding “Convolutional” layer. Convolutional Neural Networks (CNN) come from image processing. They pass a “filter” over the data and calculate a higher-level representation. They have been shown to work surprisingly well for text, even though they have none of the sequence processing ability of LSTMs.

In [59]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv 

In [60]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

Train on 1004322 samples, validate on 669548 samples
Epoch 1/3
1004322/1004322 [==============================] - 3260s - loss: 0.2953 - acc: 0.8735 - val_loss: 0.2727 - val_acc: 0.8840
Epoch 2/3
1004322/1004322 [==============================] - 3256s - loss: 0.2576 - acc: 0.8918 - val_loss: 0.2658 - val_acc: 0.8884
Epoch 3/3
1004322/1004322 [==============================] - 6413s - loss: 0.2406 - acc: 0.9000 - val_loss: 0.2677 - val_acc: 0.8884


### Save processed Data

In [28]:
df_save = pd.DataFrame(data)
df_label = pd.DataFrame(np.array(labels))

In [29]:
result = pd.concat([df_save, df_label], axis = 1)

In [31]:
result.to_csv('train_dense_word_vectors.csv', index=False)

## <b>Use pre-trained Glove word embeddings<b>

In this subsection, I want to use word embeddings from pre-trained Glove. It was trained on a dataset of one billion tokens (words) with a vocabulary of 400 thousand words. The glove has embedding vector sizes, including 50, 100, 200 and 300 dimensions. I chose the 100-dimensional version. I also want to see the model behavior in case the learned word weights do not get updated. I, therefore, set the trainable attribute for the model to be False.

### Get embeddings from Glove 

In [61]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [105]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

### Develop model

I use the same model architecture with a convolutional layer on top of the LSTM layer. 

In [106]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [107]:
model_glove.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

Train on 1004322 samples, validate on 669548 samples
Epoch 1/3
1004322/1004322 [==============================] - 2645s - loss: 0.3780 - acc: 0.8262 - val_loss: 0.3308 - val_acc: 0.8524
Epoch 2/3
1004322/1004322 [==============================] - 2090s - loss: 0.3387 - acc: 0.8484 - val_loss: 0.3222 - val_acc: 0.8598
Epoch 3/3
1004322/1004322 [==============================] - 2125s - loss: 0.3275 - acc: 0.8546 - val_loss: 0.3125 - val_acc: 0.8641


## <b>Word embedding visialization<b>

In this subsection, I want to visualize word embedding weights obtained from trained models. Word embeddings with 100 dimensions are first reduced to 2 dimensions using t-SNE. Tensorflow has an excellent tool to visualize the embeddings in a great way, but here I just want to visualize the word relationship. 

### Get embedding weights from glove

In [115]:
word_weights = model_glove.layers[0].get_weights()[0]

### Get word list 

In [137]:
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

### Scatter plot of first two components of TSNE

In [220]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(word_weights)

In [223]:
number_of_words = 1000
trace = go.Scatter(
    x = X_embedded[0:number_of_words,0], 
    y = X_embedded[0:number_of_words, 1],
    mode = 'markers',
    text= word_list[0:number_of_words]
)

layout = dict(title= 't-SNE 1 vs t-SNE 2 for sirst 1000 words ',
              yaxis = dict(title='t-SNE 2'),
              xaxis = dict(title='t-SNE 1'),
              hovermode= 'closest')

fig = dict(data = [trace], layout= layout)
py.iplot(fig)

## <b>Word Embeddings from Word2Vec<b>

In this subsection, I use word2vec to create word embeddings from the review comments. Word2vec is one algorithm for learning a word embedding from a text corpus.

In [197]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/sabber/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

###  Tokenize the reviews coments. 

In [198]:
df['tokenized'] = df.apply(lambda row : nltk.word_tokenize(row['text']), axis=1)

In [199]:
df.head()

,stars,text,tokenized
0,5,The minute I realized that Conflict was a bloc...,"[The, minute, I, realized, that, Conflict, was..."
2,5,I love Conflict Kitchen. The food is fantasti...,"[I, love, Conflict, Kitchen, ., The, food, is,..."
3,4,Holy moly! I'm addicted!\n\nI first heard of C...,"[Holy, moly, !, I, 'm, addicted, !, I, first, ..."
4,4,"Had some great Persian food, though it was mor...","[Had, some, great, Persian, food, ,, though, i..."
5,4,Yummy food. Good prices. Encourages me to try ...,"[Yummy, food, ., Good, prices, ., Encourages, ..."


### Train word2vec model

In [200]:
model_w2v = Word2Vec(df['tokenized'], size=100)

In [203]:
X = model_w2v[model_w2v.wv.vocab]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



### Plot Word Vectors Using PCA

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [209]:
tsvd = TruncatedSVD(n_components=5, n_iter=10)
result = tsvd.fit_transform(X)

In [210]:
result.shape

(158919, 5)

In [222]:
tsvd_word_list = []
words = list(model_w2v.wv.vocab)
for i, word in enumerate(words):
    tsvd_word_list.append(word)

trace = go.Scatter(
    x = result[0:number_of_words, 0], 
    y = result[0:number_of_words, 1],
    mode = 'markers',
    text= tsvd_word_list[0:number_of_words]
)

layout = dict(title= 'SVD 1 vs SVD 2',
              yaxis = dict(title='SVD 2'),
              xaxis = dict(title='SVD 1'),
              hovermode= 'closest')

fig = dict(data = [trace], layout= layout)
py.iplot(fig)